In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import glob
import json
from PIL import Image

# 학습, 검증, 테스트 데이터 경로 생성
- 새로운 경로 생성
- 이미지 및 어노테이션 파일 추출

In [2]:
origin_data_path = '/Users/kimhongseok/cv_79_projects/part2/20/origin_data'
pjt_path = '/Users/kimhongseok/cv_79_projects/part2/20'
data_path = os.path.join(pjt_path, 'data')
pjt_name = 'sec'

train_root = os.path.join(data_path, 'train')
valid_root = os.path.join(data_path, 'valid')
test_root = os.path.join(data_path, 'test')

cls_list = os.listdir(os.path.join(origin_data_path, 'images'))
cls_list.remove('.DS_Store')

for folder in [train_root, valid_root, test_root]:
    if not os.path.exists(folder):
        os.makedirs(folder)
        for s in ['images', 'labels']:
            s_folder = os.path.join(folder, s)
            if not os.path.exists(s_folder):
                os.makedirs(s_folder)

In [3]:
file_list = glob.glob(f'{origin_data_path}/images/*/*/*.png')
len(file_list)

4175

In [4]:
4175 / 5

835.0

In [5]:
# Bounding box 형태 변환: xml -> yolo

def xml_to_yolo_bbox(bbox, w, h):
    x_center = (bbox[0] + bbox[2])/2/w # w: 이미지의 가로크기 -> w로 나눠주면 normalization이 된다.
    y_center = (bbox[1] + bbox[3])/2/h
    width = (bbox[2] - bbox[0])/w
    height = (bbox[3] - bbox[1])/h

    return [x_center, y_center, width, height]

In [6]:
# xml -> txt

import xml.etree.ElementTree as ET

file_name = file_list[0].split('/')[-1].replace('png', 'xml')
save_name = file_name.replace('xml', 'txt')
file_path = f'{origin_data_path}/Annotation/Train/Pascal/Astrophysics/{file_name}'
save_path = f'{origin_data_path}/Label/{save_name}'

result = []

tree = ET.parse(file_path)
root = tree.getroot()

width = int(root.find('size').find('width').text)
height = int(root.find('size').find('height').text)

for obj in root.findall('object'):
    label = obj.find('name').text
    if label in cls_list:
        index = cls_list.index(label)
        bbox = [int(x.text) for x in obj.find('bndbox')]
        yolo_bbox = xml_to_yolo_bbox(bbox, width, height)
        bbox_string = ' '.join([str(x) for x in yolo_bbox])
        result.append(f'{index} {bbox_string}')

if result:
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(result))

In [7]:
# 멀티 쓰레드를 사용해서 빠르게 처리
import concurrent.futures
import warnings
import gc
from tqdm.notebook import tqdm

warnings.filterwarnings(action='ignore')

def process_chunk(chunk):
    print('시작')
    tbar = tqdm(chunk)
    for file in tbar:
        file_name = file.split('/')[-1].replace('png', 'xml')
        save_name = file_name.replace('xml', 'txt')
        file_path = f'{origin_data_path}/Annotation/Train/Pascal/Astrophysics/{file_name}'
        save_path = f'{origin_data_path}/Label/{save_name}'

        result = []

        tree = ET.parse(file_path)
        root = tree.getroot()

        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)

        for obj in root.findall('object'):
            label = obj.find('name').text
            if label in cls_list:
                index = cls_list.index(label)
                bbox = [int(x.text) for x in obj.find('bndbox')]
                yolo_bbox = xml_to_yolo_bbox(bbox, width, height)
                bbox_string = ' '.join([str(x) for x in yolo_bbox])
                result.append(f'{index} {bbox_string}')

        if result:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(result))

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as excutor:
    futures = []
    for start in range(0, 4175, 4175//5):
        end = start+(4175//5)
        chunk = file_list[start:end]
        future = excutor.submit(process_chunk, chunk)
        futures.append(future)
        del future
        gc.collect()

    for future in concurrent.futures.as_completed(futures):
        future.result()
        print('끝')

시작시작

시작
시작
시작


  0%|          | 0/835 [00:00<?, ?it/s]

  0%|          | 0/835 [00:00<?, ?it/s]

  0%|          | 0/835 [00:00<?, ?it/s]

  0%|          | 0/835 [00:00<?, ?it/s]

  0%|          | 0/835 [00:00<?, ?it/s]

끝
끝
끝
끝
끝


In [8]:
import shutil
import random

random.seed(74)
img_file_list = glob.glob('/Users/kimhongseok/cv_79_projects/part2/20/origin_data/images/*/*/*.png')
file_list = []

for file in img_file_list:
    label_name = file.split('/')[-1].replace('png', 'txt')
    label_path = f'{origin_data_path}/Label/{label_name}'
    
    if os.path.isfile(label_path):
        file_list.append(file)

random.shuffle(file_list)
test_ratio = 0.1
num_file = len(file_list)

test_list = file_list[:int(num_file * test_ratio)]
valid_list = file_list[:int(num_file * test_ratio):int(num_file*test_ratio)*2]
train_list = file_list[:int(num_file * test_ratio)*2:]

for i in test_list:
    label_name = i.split('/')[-1].replace('png', 'txt')
    label_path = f'{origin_data_path}/Label/{label_name}'
    shutil.copyfile(label_path, f'{test_root}/labels/{label_name}') 
    img_name = i.split('/')[-1]
    shutil.copyfile(i, f'{test_root}/images/{img_name}')

for i in valid_list:
    label_name = i.split('/')[-1].replace('png', 'txt')
    label_path = f'{origin_data_path}/Label/{label_name}'
    shutil.copyfile(label_path, f'{valid_root}/labels/{label_name}') 
    img_name = i.split('/')[-1]
    shutil.copyfile(i, f'{valid_root}/images/{img_name}')

for i in train_list:
    label_name = i.split('/')[-1].replace('png', 'txt')
    label_path = f'{origin_data_path}/Label/{label_name}'
    shutil.copyfile(label_path, f'{train_root}/labels/{label_name}') 
    img_name = i.split('/')[-1]
    shutil.copyfile(i, f'{train_root}/images/{img_name}')

# config 파일 생성

In [9]:
pjt_root = '/Users/kimhongseok/cv_79_projects/part2/20'

In [10]:
import yaml
data = dict()

data['train'] = train_root
data['val'] = valid_root
data['test'] = test_root
data['nc'] = len(cls_list)
data['names'] = cls_list

with open(f'{pjt_root}/security_screening.yaml', 'w') as f:
    yaml.dump(data, f)

# training

In [11]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 448.9/460.4 GB disk)


In [12]:
from ultralytics import YOLO

In [13]:
pjt_root = '/Users/kimhongseok/cv_79_projects/part2/20'

origin_data_path = '/Users/kimhongseok/cv_79_projects/part2/20/origin_data'
data_root = '/Users/kimhongseok/cv_79_projects/part2/20/data'
pjt_name = 'sec'

train_root = os.path.join(data_root, 'train')
valid_root = os.path.join(data_root, 'valid')
test_root = os.path.join(data_root, 'test')

In [14]:
%cd /Users/kimhongseok/cv_79_projects/part2/20

/Users/kimhongseok/cv_79_projects/part2/20


In [15]:
model = YOLO('yolov8s.pt')
results = model.train(data='security_screening.yaml', epochs=1, batch=16, imgsz=384)

New https://pypi.org/project/ultralytics/8.2.90 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=security_screening.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=384, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_tx

train: Scanning /Users/kimhongseok/cv_79_projects/part2/20/data/train/labels.cache... 834 images, 0 backgrounds, 0 corrupt: 100%|██████████| 834/834 [00:00<?, ?it/s]
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /Users/kimhongseok/cv_79_projects/part2/20/data/valid/labels.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 384 train, 384 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.597      2.889      1.086         11        384: 100%|██████████| 53/53 [04:01<00:00,  4.55s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.82it/s]


                   all          1          2       0.98          1      0.995      0.747

1 epochs completed in 0.068 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 22.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 13.93it/s]


                   all          1          2      0.981          1      0.995      0.747
        Throwing Knife          1          2      0.981          1      0.995      0.747
Speed: 0.1ms preprocess, 41.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train3


# test

In [16]:
model = YOLO('/Users/kimhongseok/cv_79_projects/part2/20/runs/detect/train3/weights/best.pt')

metrics = model.val(split='test')

print('map50-95', metrics.box.map)
print('map50', metrics.box.map50)

Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /Users/kimhongseok/cv_79_projects/part2/20/data/test/labels... 417 images, 0 backgrounds, 0 corrupt: 100%|██████████| 417/417 [00:00<00:00, 430.06it/s]

val: New cache created: /Users/kimhongseok/cv_79_projects/part2/20/data/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:58<00:00,  2.15s/it]


                   all        417        717      0.374      0.471      0.377      0.265
               Thinner         84        139      0.387      0.187      0.204      0.108
        Throwing Knife         84        142      0.254      0.296      0.183     0.0911
              ZippoOil         89        154      0.246      0.526      0.303      0.212
               Battery         75        128      0.347      0.359      0.227      0.109
                   HDD         93        154      0.635      0.987      0.969      0.804
Speed: 0.1ms preprocess, 115.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val
map50-95 0.2647477374204408
map50 0.37704859466739543


# inference

In [29]:
from ultralytics.utils.plotting import Annotator

In [18]:
%cd /Users/kimhongseok/cv_79_projects/part2/20

/Users/kimhongseok/cv_79_projects/part2/20


In [19]:
test_file_list = glob.glob('/Users/kimhongseok/cv_79_projects/part2/20/data/test/images/*')
random.shuffle(test_file_list)

In [70]:
import torchvision.transforms as T

model = YOLO('/Users/kimhongseok/cv_79_projects/part2/20/runs/detect/train3/weights/best.pt')

IMG_SIZE = (384, 384)
test_data_transforms = T.Compose([
    T.ToTensor(),
    T.Resize(IMG_SIZE)
])

color_dict = [(random.randint(0, 255) for _ in range(3)) for _ in range(len(model.names))]
print(color_dict)

[<generator object <listcomp>.<genexpr> at 0x351e698c0>, <generator object <listcomp>.<genexpr> at 0x37ab55ee0>, <generator object <listcomp>.<genexpr> at 0x37aef3680>, <generator object <listcomp>.<genexpr> at 0x37aef3a00>, <generator object <listcomp>.<genexpr> at 0x37aef3920>]


In [71]:
test_img = cv2.imread(test_file_list[0])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
results = model(test_img)


0: 224x384 1 ZippoOil, 45.6ms
Speed: 1.2ms preprocess, 45.6ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 384)


In [73]:
plt.imshow(img_src)

In [79]:
test_img = cv2.imread(test_file_list[10])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
plt.imshow(img_src)
plt.show()
results = model(img_src)[0]

annotator = Annotator(img_src)
boxes = results.boxes

for box in boxes:
    b = box.xyxy[0]
    cls = box.cls
    annotator.box_label(b, model.names[int(cls)], color_dict[int(cls)])

img_src = annotator.result()
plt.imshow(img_src)
plt.show()

<Figure size 640x480 with 1 Axes>


0: 224x384 1 ZippoOil, 38.2ms
Speed: 1.1ms preprocess, 38.2ms inference, 0.3ms postprocess per image at shape (1, 3, 224, 384)


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Scalar value for argument 'color' is not numeric
>  - Scalar value for argument 'color' is not numeric
>  - argument for rectangle() given by name ('thickness') and position (4)
>  - argument for rectangle() given by name ('thickness') and position (4)
